In [1]:
import pandas as pd
import pandas_ta as ta
from tqdm.auto import tqdm
import numpy as np
import json
import os

/Users/beilakaliev/Library/Caches/pypoetry/virtualenvs/ml-modeling-TB8BmMSm-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# --- ШАГ 1: ручная карта подтвержденных сплитов ---
def load_split_map(filepath: str) -> dict:
    """Загружает карту сплитов из JSON-файла."""
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            split_map = json.load(f)
        print(f"Карта сплитов успешно загружена из: {filepath}")
        return split_map
    except FileNotFoundError:
        print(f"Предупреждение: Файл с картой сплитов не найден по пути {filepath}. Корректировка не будет произведена.")
        return {}
    except json.JSONDecodeError:
        print(f"ОШИКА: Не удалось прочитать JSON-файл {filepath}. Проверьте его формат.")
        return {}
    

In [3]:
# ==============================================================================
# РАСШИРЕННЫЙ ФИЧАИНЖИНИРИНГ (ПОЛНАЯ ИНТЕГРИРОВАННАЯ ВЕРСИЯ)
# ==============================================================================

def add_features_extended(df: pd.DataFrame, split_map: dict):
    """
    Добавляет в DataFrame расширенный набор технических индикаторов и статистических признаков.
    """
    print("Начало расширенного фичаинжиниринга...")
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values(by=['Ticker', 'Date']).reset_index(drop=True)

    # --- 1. Признаки тренда (Trend Features) ---
    print("Расчет индикаторов тренда...")
    sma_periods = [3, 5, 7, 10, 15, 20, 30, 40, 50, 70, 100, 150, 200]
    for i in sma_periods:
        df[f"sma_{i}"] = df.groupby('Ticker')['Close'].transform(lambda x: x.rolling(i).mean())

    macd = df.groupby('Ticker', group_keys=False).apply(lambda x: ta.macd(x['Close'], fast=12, slow=26, signal=9))
    df = pd.concat([df, macd], axis=1)
    adx = df.groupby('Ticker', group_keys=False).apply(lambda x: ta.adx(x['High'], x['Low'], x['Close'], length=14))
    df = pd.concat([df, adx], axis=1)

    # --- 2. Признаки моментума (Momentum Features) ---
    print("Расчет индикаторов моментума...")
    rsi_periods = [5, 7, 14, 21, 30, 50]
    for i in rsi_periods:
        df[f"rsi_{i}"] = df.groupby('Ticker')['Close'].transform(lambda x: ta.rsi(x, length=i))

    stoch = df.groupby('Ticker', group_keys=False).apply(lambda x: ta.stoch(x['High'], x['Low'], x['Close'], k=14, d=3, smooth_k=3))
    df = pd.concat([df, stoch], axis=1)
    willr_periods = [5, 7, 14, 21, 30]
    for i in willr_periods:
        df[f"willr_{i}"] = df.groupby('Ticker', group_keys=False).apply(lambda x: ta.willr(x['High'], x['Low'], x['Close'], length=i))

    # --- 3. Признаки волатильности (Volatility Features) ---
    print("Расчет индикаторов волатильности...")
    atr_periods = [5, 7, 14, 21]
    for i in atr_periods:
        atr = df.groupby('Ticker', group_keys=False).apply(lambda x: ta.atr(x['High'], x['Low'], x['Close'], length=i))
        df[f"atr_{i}"] = atr

    bollinger = df.groupby('Ticker', group_keys=False).apply(lambda x: ta.bbands(x['Close'], length=20, std=2))
    
    df = pd.concat([df, bollinger], axis=1)
    df['bb_width_norm'] = (df['BBU_20_2.0_2.0'] - df['BBL_20_2.0_2.0']) / (df['BBM_20_2.0_2.0'] + 1e-9)

    # --- 4. Признаки объема (Volume Features) ---
    print("Расчет индикаторов объема...")
    vol_sma_periods = [5, 7, 14, 20, 30]
    for i in vol_sma_periods:
        df[f"vol_sma_{i}"] = df.groupby('Ticker')['Volume'].transform(lambda x: x.rolling(i).mean())
        df[f'relative_volume_{i}'] = df['Volume'] / (df[f"vol_sma_{i}"] + 1e-9)

    print("Расчет и нормализация OBV...")
    obv_series = df.groupby('Ticker', group_keys=False).apply(lambda x: ta.obv(x['Close'], x['Volume']))
    df['obv'] = obv_series 
    for i in vol_sma_periods:
        obv_sma_col = f'obv_sma_{i}'
        df[obv_sma_col] = df.groupby('Ticker')['obv'].transform(lambda x: x.rolling(i).mean())
        df[f'obv_relative_{i}'] = df['obv'] / (df[obv_sma_col] + 1e-9)        
        df[f'obv_trend_{i}'] = df['obv'] - df[obv_sma_col]


    df['turnover'] = df['Close'] * df['Volume']
    for i in vol_sma_periods: # Используем те же периоды для сопоставимости
        df[f"turnover_sma_{i}"] = df.groupby('Ticker')['turnover'].transform(lambda x: x.rolling(i).mean())
        df[f'relative_turnover_{i}'] = df['turnover'] / (df[f"turnover_sma_{i}"] + 1e-9)
    df.drop(columns=['turnover'], inplace=True)

    # --- 5. Признаки свечей и меж-дневной динамики ---
    print("Расчет признаков свечей и меж-дневной динамики...")
    df['day_range_norm'] = (df['High'] - df['Low']) / (df['Close'] + 1e-9)
    df['intraday_move_norm'] = (df['Close'] - df['Open']) / (df['Close'] + 1e-9)
    df['upper_wick_norm'] = (df['High'] - df[['Open', 'Close']].max(axis=1)) / (df['Close'] + 1e-9)
    df['lower_wick_norm'] = (df[['Open', 'Close']].min(axis=1) - df['Low']) / (df['Close'] + 1e-9)
    df['overnight_gap_norm'] = (df['Open'] - df.groupby('Ticker')['Close'].shift(1)) / (df.groupby('Ticker')['Close'].shift(1) + 1e-9)
    daily_range = df['High'] - df['Low']
    df['range_expansion_ratio'] = daily_range / (df.groupby('Ticker')['High'].shift(1) - df.groupby('Ticker')['Low'].shift(1) + 1e-9)

    # --- 6. Признаки взаимодействия Цены и Объема ---
    print("Расчет признаков взаимодействия Цены и Объема...")
    # Используем relative_volume_20, так как он соответствует периоду Bollinger Bands
    df['volume_weighted_move'] = df['intraday_move_norm'] * df['relative_volume_20']
    df['daily_return'] = df.groupby('Ticker')['Close'].pct_change()
    df['up_day_volume'] = df.apply(lambda row: row['Volume'] if row['daily_return'] > 0 else 0, axis=1)
    df['down_day_volume'] = df.apply(lambda row: row['Volume'] if row['daily_return'] <= 0 else 0, axis=1)
    
    sma_up_vol = df.groupby('Ticker')['up_day_volume'].transform(lambda x: x.rolling(20).mean())
    sma_down_vol = df.groupby('Ticker')['down_day_volume'].transform(lambda x: x.rolling(20).mean())
    df['up_down_volume_ratio'] = sma_up_vol / (sma_down_vol + 1e-9)
    df.drop(columns=['daily_return', 'up_day_volume', 'down_day_volume'], inplace=True)

    # --- 7. Статистические признаки ---
    print("Расчет статистических признаков...")
    df['log_return'] = df.groupby('Ticker')['Close'].transform(lambda x: np.log(x / x.shift(1)))
    stat_periods = [7, 14, 21]
    for i in stat_periods:
        df[f'rolling_std_{i}'] = df.groupby('Ticker')['Close'].transform(lambda x: x.rolling(i).std())
        df[f'rolling_skew_{i}'] = df.groupby('Ticker')['log_return'].transform(lambda x: x.rolling(i).skew())
        df[f'rolling_kurt_{i}'] = df.groupby('Ticker')['log_return'].transform(lambda x: x.rolling(i).kurt())

    # --- 8. Календарные признаки ---
    print("Добавление календарных признаков...")
    df['day_of_week'] = df['Date'].dt.dayofweek
    df['month'] = df['Date'].dt.month
    df['week_of_year'] = df['Date'].dt.isocalendar().week.astype(int)
    df['day_of_year'] = df['Date'].dt.dayofyear
    df['quarter'] = df['Date'].dt.quarter
    df['is_month_start'] = df['Date'].dt.is_month_start.astype(int)
    df['is_month_end'] = df['Date'].dt.is_month_end.astype(int)
    df['is_quarter_start'] = df['Date'].dt.is_quarter_start.astype(int)
    df['is_quarter_end'] = df['Date'].dt.is_quarter_end.astype(int)
    df['is_year_start'] = df['Date'].dt.is_year_start.astype(int)
    df['is_year_end'] = df['Date'].dt.is_year_end.astype(int)
    def _get_season(month):
        if month in [12, 1, 2]: return 0 # Winter
        elif month in [3, 4, 5]: return 1 # Spring
        elif month in [6, 7, 8]: return 2 # Summer
        else: return 3 # Autumn
    df['season'] = df['month'].apply(_get_season)
    
    # --- 9. Событийные признаки и взаимодействия ---
    print("Расчет признаков взаимодействия и событий...")
    # Ваш полный набор признаков взаимодействия
    for i in sma_periods:
        sma_col = f'sma_{i}'
        df[f'close_to_{sma_col}'] = (df['Close'] - df[sma_col]) / (df[sma_col] + 1e-9)
        df[f'high_to_{sma_col}'] = (df['High'] - df[sma_col]) / (df[sma_col] + 1e-9)
        df[f'low_to_{sma_col}'] = (df['Low'] - df[sma_col]) / (df[sma_col] + 1e-9)
        df[f'open_to_{sma_col}'] = (df['Open'] - df[sma_col]) / (df[sma_col] + 1e-9)

    # Признак сплита (событийный)
    df['is_split_day'] = 0
    for ticker, events in split_map.items():
        for event in events:
            event_date = pd.to_datetime(event['date'])
            idx = df[(df['Ticker'] == ticker) & (df['Date'] == event_date)].index
            if not idx.empty:
                df.loc[idx, 'is_split_day'] = 1
    print(f"Найдено и отмечено {df['is_split_day'].sum()} дней со сплитами.")


    # --- НОВЫЙ РАЗДЕЛ 10: ПРИЗНАКИ ПЕРЕСЕЧЕНИЯ СКОЛЬЗЯЩИХ СРЕДНИХ ---
    print("Расчет признаков пересечения скользящих средних...")

        
    # Определяем пары для анализа (быстрая, медленная)
    sma_cross_pairs = [
        (70, 200), # Классическое "Золотое/Мертвое" пересечение
        (50, 200), 
        (20, 50),  # Среднесрочное пересечение
        # Краткосрочные пересечения
        (7, 15),
        (3, 10),
        (3, 7),
    ]

    for fast_period, slow_period in sma_cross_pairs:
        fast_col = f'sma_{fast_period}'
        slow_col = f'sma_{slow_period}'
        
        # Убедимся, что нужные SMA уже посчитаны
        if fast_col not in df.columns or slow_col not in df.columns:
            raise ValueError(f"NO sma {fast_col = }. {slow_col = }")
    
        # --- Признак 2: Состояние тренда ---
        state_col = f'sma{fast_period}_above_sma{slow_period}'
        df[state_col] = (df[fast_col] > df[slow_col]).astype(int)
        
        # --- Признак 1: Сигнал пересечения ---
        signal_col = f'sma{fast_period}_cross_sma{slow_period}'
        # Сдвигаем состояние на 1 день назад, чтобы сравнить "сегодня" и "вчера"
        prev_state = df.groupby('Ticker')[state_col].shift(1)
        # Пересечение - это когда состояние изменилось (0->1 или 1->0)
        df[signal_col] = 0
        # Бычье пересечение (+1): было 0, стало 1
        df.loc[(df[state_col] == 1) & (prev_state == 0), signal_col] = 1
        # Медвежье пересечение (-1): было 1, стало 0
        df.loc[(df[state_col] == 0) & (prev_state == 1), signal_col] = -1

        # --- Признак 3: Дни с момента пересечения ---
        days_since_col = f'days_since_sma{fast_period}_cross_{slow_period}'
        # Находим, где были пересечения (не равно 0)
        cross_events = df[signal_col].ne(0)
        # Создаем группы, которые начинаются с каждого пересечения
        cross_groups = cross_events.cumsum()
        # Считаем дни внутри каждой группы
        df[days_since_col] = df.groupby(['Ticker', cross_groups]).cumcount()


    # --- 11. Продвинутые сигналы технического анализа ---
    print("Расчет продвинутых сигналов теханализа...")

    # 1. Сигналы ADX (Average Directional Index)
    # Что это: ADX показывает СИЛУ тренда (не направление). Пересечение линий +DI и -DI показывает НАПРАВЛЕНИЕ.    
    adx_col = 'ADX_14'
    dmp_col = 'DMP_14' # +DI
    dmn_col = 'DMN_14' # -DI
    
    # Убедимся, что колонки существуют
    if not (adx_col in df.columns and dmp_col in df.columns and dmn_col in df.columns):
        raise ValueError(f"ERROR no {adx_col = }. {dmp_col = }, { dmn_col = }")
        # Признак "Сила направленного движения": ADX, умноженный на знак тренда.
        # Знак тренда = +1, если +DI выше -DI (бычий), и -1, если наоборот.
    trend_direction = (df[dmp_col] > df[dmn_col]).astype(int) * 2 - 1 # Преобразует True/False в +1/-1
    df['adx_trend_strength'] = df[adx_col] * trend_direction

    # 2. Сигналы MACD (Moving Average Convergence Divergence)
    # Что это: Пересечение линии MACD с ее сигнальной линией - классический сигнал.
    # Гистограмма (разница между линиями) показывает силу моментума.
    macd_line_col = 'MACD_12_26_9'
    signal_line_col = 'MACDs_12_26_9'
    hist_col = 'MACDh_12_26_9'

    if not (macd_line_col in df.columns and signal_line_col in df.columns):
        raise ValueError(f"NO {macd_line_col = }. {signal_line_col = }")

    # Состояние MACD: +1 если MACD выше сигнальной линии (бычье), -1 если ниже (медвежье)
    df['macd_state'] = (df[macd_line_col] > df[signal_line_col]).astype(int) * 2 - 1
    
    # Сигнал пересечения MACD (+1 = бычье, -1 = медвежье)
    prev_macd_state = df.groupby('Ticker')['macd_state'].shift(1)
    df['macd_cross_signal'] = 0
    df.loc[(df['macd_state'] == 1) & (prev_macd_state == -1), 'macd_cross_signal'] = 1
    df.loc[(df['macd_state'] == -1) & (prev_macd_state == 1), 'macd_cross_signal'] = -1
    
    # Признак "Ускорение моментума": растет ли гистограмма?
    df['macd_hist_acceleration'] = (df[hist_col] > df.groupby('Ticker')[hist_col].shift(1)).astype(int)


    # 3. Сигналы по Полосам Боллинджера (Bollinger Bands)
    # Что это: Касание или пробой границ канала - сильный сигнал.
    upper_bb_col = 'BBU_20_2.0_2.0'
    lower_bb_col = 'BBL_20_2.0_2.0'
    
    if not (upper_bb_col in df.columns and lower_bb_col in df.columns):
        raise ValueError(f"NO { upper_bb_col =}. {lower_bb_col = }")
    # Признак "Пробой верхней границы"
    df['bb_upper_breakout'] = (df['Close'] > df[upper_bb_col]).astype(int)
    # Признак "Пробой нижней границы"
    df['bb_lower_breakout'] = (df['Close'] < df[lower_bb_col]).astype(int)
    # Положение цены внутри канала (от 0 до 1)
    # 0 = на нижней границе, 1 = на верхней границе, >1 = пробой вверх, <0 = пробой вниз
    df['bb_percent_b'] = (df['Close'] - df[lower_bb_col]) / (df[upper_bb_col] - df[lower_bb_col] + 1e-9)


    print("Генерация признаков на основе ставки ЦБ...")    
    if 'cbr_rate' in df.columns:
        # 1. Величина изменения ставки (рассчитывается внутри каждой группы тикеров)
        # .transform() применяет операцию к группе и возвращает результат того же размера,
        # что и исходный DataFrame, избегая смешивания данных.
        rate_change = df.groupby('Ticker')['cbr_rate'].transform(lambda x: x.replace(-1, np.nan).diff())        
        df['cbr_rate_change_value'] = rate_change.fillna(0)

        # 2. Факт изменения ставки (1 - было изменение, 0 - не было)
        df['cbr_rate_change_flag'] = (df['cbr_rate_change_value'] != 0).astype(int)
        
    else:
        print("Предупреждение: Колонка 'cbr_rate' не найдена. Признаки на ее основе не будут созданы.")
        raise Exception


    # ---  Признаки Моментума и Относительной Силы ---
    print("Расчет признаков моментума и относительной силы...")

    # Периоды для анализа
    momentum_periods = [3, 5, 7, 10, 14, 21, 30, 60, 100]

    for n in momentum_periods:
        # Рассчитывается внутри каждого тикера
        df[f'momentum_{n}d'] = df.groupby('Ticker')['Close'].transform(
            lambda x: x.pct_change(periods=n)
        )        
        # --- Моментум, скорректированный на риск (Sharpe Ratio тренда) ---
        # log_return уже должен быть рассчитан в секции статистических признаков
        if not ('log_return' in df.columns):
            raise ValueError("No log_return")
        returns_grouped = df.groupby('Ticker')['log_return']
        mean_returns = returns_grouped.transform(lambda x: x.rolling(n).mean())
        std_returns = returns_grouped.transform(lambda x: x.rolling(n).std())
        df[f'momentum_sharpe_{n}d'] = mean_returns / (std_returns + 1e-9)

    # ---: Кросс-секционный моментум (Ранжирование) ---
    # Этот расчет должен идти после цикла, так как он работает со всеми тикерами одновременно
    # для каждой конкретной даты.
    print("Расчет кросс-секционного ранжирования по моментуму...")
    for n in momentum_periods:
        # groupby('Date') - ключевой шаг. Ранжируем акции ВНУТРИ каждого дня.
        # rank(pct=True) - преобразует ранг в процентиль (от 0.0 до 1.0), 
        # что является лучшей практикой для ML моделей.
        df[f'momentum_rank_{n}d'] = df.groupby('Date')[f'momentum_{n}d'].rank(pct=True)




    # --- 10. Финальная очистка от NaN ---
    print("Очистка данных от NaN...")
    # Находим самый длинный период из всех использованных
    longest_period = max(sma_periods)
    print(f"Удаление первых {longest_period} строк для каждого тикера для прогрева индикаторов...")
    # Отбрасываем N первых строк для КАЖДОГО тикера
    df = df.groupby('Ticker', group_keys=False).apply(lambda x: x.iloc[longest_period:])
    # Дополнительно убираем строки, если где-то остались NaN (например, из-за .shift() в новых признаках)
    df.dropna(inplace=True)
    
    print("Расширенный фичаинжиниринг завершен.")
    return df.reset_index(drop=True)

In [4]:

data_folder = "../data/"
normalized_data_filename = 'moex_normalized_data.csv'
features_data_filename = 'moex_with_features.csv'

config_folder = "config/"
split_map_filename = os.path.join(config_folder, "splits.json")

exclude_list_filename = os.path.join(config_folder, "feature_exclude_list.json") # Путь к "черному списку"

# --- ШАГ 1: Загрузка сырых данных ---
print(f"Загрузка сырых данных из: {data_folder + normalized_data_filename}")
try:
    raw_data = pd.read_csv(data_folder + normalized_data_filename)
    print("Данные успешно загружены.")
except FileNotFoundError:
    print(f"ОШИБКА: Файл не найден. Убедитесь, что скрипт сохранения сырых данных был запущен.")
    exit()

split_map = load_split_map(split_map_filename)

# --- ШАГ 2: Добавление признаков ---
# tqdm.pandas(desc="Расчет индикаторов")
# data_with_features = add_features(raw_data)
data_with_features = add_features_extended(raw_data, split_map)



# --- ЭТАП 2: ФИЛЬТРАЦИЯ ПО "ЧЕРНОМУ СПИСКУ" ---
print("\n--- Фильтрация признаков по списку исключений ---")

features_to_exclude = []
try:
    with open(exclude_list_filename, 'r', encoding='utf-8') as f:
        features_to_exclude = json.load(f)
    print(f"Загружен список из {len(features_to_exclude)} признаков для исключения.")
except FileNotFoundError:
    print(f"ИНФО: Файл исключений '{exclude_list_filename}' не найден. Все признаки будут сохранены.")
except json.JSONDecodeError:
    print(f"ПРЕДУПРЕЖДЕНИЕ: Не удалось прочитать JSON из '{exclude_list_filename}'. Все признаки будут сохранены.")

if features_to_exclude:
    # Находим, какие из признаков в списке реально есть в DataFrame
    cols_to_drop = [col for col in features_to_exclude if col in data_with_features.columns]
    
    if cols_to_drop:
        print(f"Будет исключено {len(cols_to_drop)} признаков: {cols_to_drop}")
        final_df = data_with_features.drop(columns=cols_to_drop)
    else:
        print("Ни один из признаков в списке исключений не найден в DataFrame.")
        final_df = data_with_features
else:
    final_df = data_with_features




print("\n--- DataFrame с признаками ---")
print(final_df.info())





# --- ШАГ 3: Сохранение результата ---
print(f"\nСохранение данных с признаками в файл: {data_folder + features_data_filename}")
final_df.to_csv(data_folder + features_data_filename, index=False)

print("Скрипт фичаинжиниринга выполнен успешно!")

Загрузка сырых данных из: ../data/moex_normalized_data.csv
Данные успешно загружены.
Карта сплитов успешно загружена из: config/splits.json
Начало расширенного фичаинжиниринга...
Расчет индикаторов тренда...


/var/folders/lh/9k36l4vn35v4qnj9z669pmtc0000gn/T/ipykernel_68944/3810807164.py:19: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  macd = df.groupby('Ticker', group_keys=False).apply(lambda x: ta.macd(x['Close'], fast=12, slow=26, signal=9))
/var/folders/lh/9k36l4vn35v4qnj9z669pmtc0000gn/T/ipykernel_68944/3810807164.py:21: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  adx = df.groupby('Ticker', group_keys=False).apply(

Расчет индикаторов моментума...


/var/folders/lh/9k36l4vn35v4qnj9z669pmtc0000gn/T/ipykernel_68944/3810807164.py:30: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  stoch = df.groupby('Ticker', group_keys=False).apply(lambda x: ta.stoch(x['High'], x['Low'], x['Close'], k=14, d=3, smooth_k=3))
/var/folders/lh/9k36l4vn35v4qnj9z669pmtc0000gn/T/ipykernel_68944/3810807164.py:34: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df[f"willr_{i}"] = df.groupby('Ti

Расчет индикаторов волатильности...


...

Расчет кросс-секционного ранжирования по моментуму...


/var/folders/lh/9k36l4vn35v4qnj9z669pmtc0000gn/T/ipykernel_68944/3810807164.py:292: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'momentum_rank_{n}d'] = df.groupby('Date')[f'momentum_{n}d'].rank(pct=True)
/var/folders/lh/9k36l4vn35v4qnj9z669pmtc0000gn/T/ipykernel_68944/3810807164.py:292: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'momentum_rank_{n}d'] = df.groupby('Date')[f'momentum_{n}d'].rank(pct=True)
/var/folders/lh/9k36l4vn35v4qnj9z669pmtc0000gn/T/ipykernel_68944/3810807164.py:292: PerformanceWarning: DataFrame 

Очистка данных от NaN...
Удаление первых 200 строк для каждого тикера для прогрева индикаторов...


/var/folders/lh/9k36l4vn35v4qnj9z669pmtc0000gn/T/ipykernel_68944/3810807164.py:303: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('Ticker', group_keys=False).apply(lambda x: x.iloc[longest_period:])


Расширенный фичаинжиниринг завершен.

--- Фильтрация признаков по списку исключений ---
Загружен список из 5 признаков для исключения.
Будет исключено 5 признаков: ['is_year_start', 'is_year_end', 'is_split_day', 'obv', 'cbr_rate']

--- DataFrame с признаками ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173989 entries, 0 to 173988
Columns: 220 entries, Date to momentum_rank_100d
dtypes: datetime64[ns](1), float64(184), int32(4), int64(30), object(1)
memory usage: 289.4+ MB
None

Сохранение данных с признаками в файл: ../data/moex_with_features.csv
Скрипт фичаинжиниринга выполнен успешно!


In [5]:
data_with_features

,Date,Ticker,Open,High,Low,Close,Volume,cbr_rate,sma_3,sma_5,...,momentum_sharpe_100d,momentum_rank_3d,momentum_rank_5d,momentum_rank_7d,momentum_rank_10d,momentum_rank_14d,momentum_rank_21d,momentum_rank_30d,momentum_rank_60d,momentum_rank_100d
0,2015-03-27,AFKS,17.04,17.30,16.62,17.30,9700300.0,14.0,17.313333,17.084,...,0.024654,0.872727,0.890909,0.890909,1.000000,0.927273,0.672727,0.636364,0.872727,0.618182
1,2015-03-30,AFKS,17.21,17.47,17.08,17.40,13542700.0,14.0,17.246667,17.246,...,0.012765,0.254545,0.909091,0.872727,0.981818,1.000000,0.618182,0.490909,0.909091,0.490909
2,2015-03-31,AFKS,17.37,17.93,17.10,17.90,17961200.0,14.0,17.533333,17.448,...,0.013097,0.781818,0.909091,0.927273,0.927273,1.000000,0.672727,0.527273,0.927273,0.509091
3,2015-04-01,AFKS,17.90,18.02,17.52,17.75,12653600.0,14.0,17.683333,17.478,...,0.017035,0.418182,0.381818,0.872727,0.872727,0.981818,0.654545,0.109091,0.909091,0.490909
4,2015-04-02,AFKS,17.75,18.00,17.12,17.85,21043800.0,14.0,17.833333,17.640,...,0.019174,0.500000,0.509091,0.690909,0.818182,0.981818,0.727273,0.400000,0.836364,0.527273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173984,2025-09-12,YDEX,4312.00,4312.00,4171.50,4200.00,673681.0,18.0,4273.500000,4320.100,...,-0.022856,0.410256,0.346154,0.397436,0.641026,0.410256,0.756410,0.538462,0.653846,0.628205
173985,2025-09-15,YDEX,4220.50,4222.00,4120.00,4150.00,473297.0,18.0,4215.666667,4269.100,...,-0.019336,0.448718,0.282051,0.435897,0.435897,0.397436,0.628205,0.525641,0.628205,0.641026
173986,2025-09-16,YDEX,4157.50,4183.00,4058.50,4127.50,493769.0,18.0,4159.166667,4219.600,...,-0.023737,0.474359,0.371795,0.384615,0.346154,0.615385,0.435897,0.538462,0.576923,0.653846
173987,2025-09-17,YDEX,4128.00,4164.00,4100.00,4161.00,448318.0,18.0,4146.166667,4187.100,...,-0.029947,0.551282,0.512821,0.397436,0.474359,0.487179,0.551282,0.525641,0.551282,0.679487


In [7]:
print("!!! All columns")

for i, name in enumerate(data_with_features.columns):
    print(i, name)

!!! All columns
0 Date
1 Ticker
2 Open
3 High
4 Low
5 Close
6 Volume
7 cbr_rate
8 sma_3
9 sma_5
10 sma_7
11 sma_10
12 sma_15
13 sma_20
14 sma_30
15 sma_40
16 sma_50
17 sma_70
18 sma_100
19 sma_150
20 sma_200
21 MACD_12_26_9
22 MACDh_12_26_9
23 MACDs_12_26_9
24 ADX_14
25 ADXR_14_2
26 DMP_14
27 DMN_14
28 rsi_5
29 rsi_7
30 rsi_14
31 rsi_21
32 rsi_30
33 rsi_50
34 STOCHk_14_3_3
35 STOCHd_14_3_3
36 STOCHh_14_3_3
37 willr_5
38 willr_7
39 willr_14
40 willr_21
41 willr_30
42 atr_5
43 atr_7
44 atr_14
45 atr_21
46 BBL_20_2.0_2.0
47 BBM_20_2.0_2.0
48 BBU_20_2.0_2.0
49 BBB_20_2.0_2.0
50 BBP_20_2.0_2.0
51 bb_width_norm
52 vol_sma_5
53 relative_volume_5
54 vol_sma_7
55 relative_volume_7
56 vol_sma_14
57 relative_volume_14
58 vol_sma_20
59 relative_volume_20
60 vol_sma_30
61 relative_volume_30
62 obv
63 obv_sma_5
64 obv_relative_5
65 obv_trend_5
66 obv_sma_7
67 obv_relative_7
68 obv_trend_7
69 obv_sma_14
70 obv_relative_14
71 obv_trend_14
72 obv_sma_20
73 obv_relative_20
74 obv_trend_20
75 obv_sma_30
